In [1]:
!pip install scikit-learn

In [39]:
import pandas as pd

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

import spacy


In [30]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train = train.dropna()

In [31]:
print(train.shape)
train.head()

(2476, 7)


,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0


In [32]:
print(test.shape)
test.head()

(288, 6)


,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [133]:
# pick features
features = train.columns.drop(['id','category'])
target = 'category'

# train and test
X_train = train[features]
X_test = test[features]

y_train = train[target]

encoder = OrdinalEncoder()

X_train['author'] = encoder.fit_transform(train['author'].values.reshape(-1,1))
X_test['author'] = encoder.transform(test['author'].values.reshape(-1,1))


In [134]:
# encoder = OrdinalEncoder()

# X_train['author'] = encoder.fit_transform(train['author'].values.reshape(-1,1))
# X_test['author'] = encoder.transform(test['author'].values.reshape(-1,1))


//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [135]:
print(X_train.shape)
X_train.head()

(2476, 5)


,author,description,price,ratingValue,pert_alcohol
0,8.0,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50
1,1.0,There have been some legendary Bowmores from t...,13500.0,97,42.90
2,8.0,This bottling celebrates master distiller Park...,150.0,97,50.00
3,8.0,What impresses me most is how this whisky evol...,4500.0,97,40.50
5,4.0,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49


In [72]:
# description

nlp = spacy.load('en_core_web_md')

def tokenize(doc):
    d = nlp(doc)
    tokens = []
    
    # You can extract anything you want from Spacy to use as a feature
    # Noun Chunks aren't the end all be all.
    # Try Extracting just tokens, or just adjectives, or just pronouns
    for chunk in d.noun_chunks:
        tokens.append(chunk.lemma_)
        
    return tokens

vect = CountVectorizer(analyzer=tokenize, max_df=7, min_df=4)

In [73]:
vect.fit(X_train['description'])

CountVectorizer(analyzer=<function tokenize at 0x1a2655e268>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=7,
        max_features=None, min_df=4, ngram_range=(1, 1), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [136]:
X = vect.transform(X_train['description'])
X = X.todense()
X

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [138]:
len(X)

2476

In [164]:
X_train3 = X_train
X_train3['id'] = list(range(2476))
matrix_df2 = matrix_df
matrix_df2['id'] = list(range(2476))

print(X_train3.shape, matrix_df2.shape)


X_train5 = X_train3.merge(matrix_df2, on='id', how='inner')

print(X_train5.shape)


(2476, 6) (2476, 28)
(2476, 33)


//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [167]:
X_train6 = X_train5.drop(columns=['id', 'description'])
X_train6.head()

,author,price,ratingValue,pert_alcohol,use barrel,vanilla ice cream,vanilla icing,vanilla note,vanilla pod,vanilla toffee,...,wax,wax fruit,wemyss malts’,wet slate,white grape,wild turkey,wine,wood polish,wood resin,wood shaving
0,8.0,85.0,97,51.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,13500.0,97,42.90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.0,150.0,97,50.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8.0,4500.0,97,40.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,150.0,96,54.49,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
X_train6.tail()

,author,price,ratingValue,pert_alcohol,use barrel,vanilla ice cream,vanilla icing,vanilla note,vanilla pod,vanilla toffee,...,wax,wax fruit,wemyss malts’,wet slate,white grape,wild turkey,wine,wood polish,wood resin,wood shaving
2471,8.0,80.0,70,57.1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2472,2.0,17.0,65,44.9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2473,6.0,35.0,65,40.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2474,8.0,60.0,63,45.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2475,4.0,30.0,60,45.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
rfc.fit(X_train6, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [170]:
test_X = vect.transform(X_test['description'])
test_X = test_X.todense()
test_matrix_df = pd.DataFrame(test_X, columns=vect.get_feature_names())
test_matrix_df = test_matrix_df[columns]
test_matrix_df['id'] = list(range(288))
assert len(test_matrix_df) == len(X_test)

In [212]:
X_test2 = X_test
X_test2['id'] = list(range(288))
X_test3 = X_test2.merge(test_matrix_df, on='id', how='inner')
X_test3.shape

(288, 33)

In [213]:
X_test4 = X_test3.drop(columns=['id','description'])
X_test5 = X_test4
X_test5['price'] = X_test5['price'].fillna(method= 'ffill')
X_test5['pert_alcohol'] = X_test5['pert_alcohol'].fillna(method= 'ffill')
print(X_test5.shape)
X_test5.tail()


(288, 31)


,author,price,ratingValue,pert_alcohol,use barrel,vanilla ice cream,vanilla icing,vanilla note,vanilla pod,vanilla toffee,...,wax,wax fruit,wemyss malts’,wet slate,white grape,wild turkey,wine,wood polish,wood resin,wood shaving
283,8.0,80.0,87,55.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
284,1.0,78.0,84,55.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
285,7.0,85.0,79,50.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
286,4.0,75.0,86,63.7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
287,8.0,35.0,86,45.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
X_test5.dtypes

author               float64
price                float64
ratingValue            int64
pert_alcohol         float64
use barrel             int64
vanilla ice cream      int64
vanilla icing          int64
vanilla note           int64
vanilla pod            int64
vanilla toffee         int64
very fruity            int64
vibrant spice          int64
vintage                int64
violet candy           int64
virgin oak             int64
virgin oak cask        int64
virginia               int64
warm cereal            int64
warm spice             int64
warmth                 int64
wave                   int64
wax                    int64
wax fruit              int64
wemyss malts’          int64
wet slate              int64
white grape            int64
wild turkey            int64
wine                   int64
wood polish            int64
wood resin             int64
wood shaving           int64
dtype: object

In [217]:
# make predictions
y_preds = rfc.predict(X_test5)
y_preds = [int(i) for i in y_preds]
assert len(y_preds) == len(test)

# create submission csv

sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['category'] = y_preds
sample_submission.to_csv('team_whiskey_submission.csv', index=False)

In [218]:
sample_submission.head()

,id,category
0,955,2
1,3532,3
2,1390,4
3,1024,1
4,1902,1


In [219]:
# pick features
features = train.columns.drop(['id','category'])
target = 'category'

# train and test
X_train = train[features]
X_test = test[features]

y_train = train[target]

encoder = OrdinalEncoder()

X_train['author'] = encoder.fit_transform(train['author'].values.reshape(-1,1))
X_test['author'] = encoder.transform(test['author'].values.reshape(-1,1))

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [220]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [221]:
punctuations = string.punctuation

In [222]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [223]:
parser = English()

In [224]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens

In [225]:
def clean_text(text):
    return text.strip().lower()

In [228]:
bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))

In [229]:
tfidf_vector = CountVectorizer(tokenizer=spacy_tokenizer)

In [230]:
from sklearn.base import TransformerMixin

In [231]:
class predictors(TransformerMixin):
    def transform(self,X,**transform_params):
        return [clean_text(text) for text in X]
    def fit(self,X,y=None,**fit_params):
        return self
    def get_params(self,deep=True):
        return {}

In [283]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
cleaner = predictors()
bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))


train_desc = X_train['description']

train_desc2 = cleaner.fit_transform(train_desc)
train_desc3 = bow_vector.fit_transform(train_desc2)

In [250]:
assert train_desc3.shape[0] == len(train)
train_desc4 = train_desc3.todense()

matrix_df = pd.DataFrame(train_desc4, columns=bow_vector.get_feature_names())

In [252]:
print(len(matrix_df))
matrix_df.head()

2476


,'s,--,-and,-at,-especially,-with,.-,...,.36,005,...,’98,’fiddich,’rothes,’s,“,”,•,…,€,﻿1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
X_train2 = X_train
X_train2['id'] = list(range(2476))
matrix_df2 = matrix_df
matrix_df2['id'] = list(range(2476))

print(X_train2.shape, matrix_df2.shape)


X_train3 = X_train2.merge(matrix_df2, on='id', how='inner')
X_train4 = X_train3.drop(columns=['description','id'])
assert len(X_train4) == len(train)

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(2476, 6) (2476, 7214)


In [263]:
X_train4.head()


,author,price_x,ratingValue,pert_alcohol,'s,--,-and,-at,-especially,-with,...,’98,’fiddich,’rothes,’s,“,”,•,…,€,﻿1
0,8.0,85.0,97,51.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,13500.0,97,42.90,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,8.0,150.0,97,50.00,0,1,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
3,8.0,4500.0,97,40.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,150.0,96,54.49,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [287]:
classifier.fit(X_train4, y_train)

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [292]:
# rfc.fit(X_train4, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [269]:
X_test.columns.tolist()

['author', 'description', 'price', 'ratingValue', 'pert_alcohol', 'id']

In [284]:
test_desc = X_test['description']

test_desc2 = cleaner.transform(test_desc)
test_desc3 = bow_vector.transform(test_desc2)

assert test_desc3.shape[0] == len(test)
test_desc4 = test_desc3.todense()

test_matrix_df = pd.DataFrame(test_desc4, columns=bow_vector.get_feature_names())


X_test2 = X_test
X_test2['id'] = list(range(288))
test_matrix_df2 = test_matrix_df
test_matrix_df2['id'] = list(range(288))


X_test3 = X_test2.merge(test_matrix_df2, on='id', how='inner')
X_test4 = X_test3.drop(columns=['description','id'])
X_test4.head()


# fill na
X_test5 = X_test4
X_test5['price_x'] = X_test4['price_x'].fillna(method= 'ffill')
X_test5['pert_alcohol'] = X_test4['pert_alcohol'].fillna(method= 'ffill')
assert len(X_test4) == len(test)

In [285]:
print(X_test5.shape)
X_test5.head()

(288, 7217)


,author,price_x,ratingValue,pert_alcohol,'s,--,-and,-at,-especially,-with,...,’98,’fiddich,’rothes,’s,“,”,•,…,€,﻿1
0,4.0,36.0,90,50.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,90.0,82,49.3,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,2.0,48.0,89,45.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5.0,180.0,90,55.8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.0,71.0,87,45.9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [289]:
y_preds = classifier.predict(X_test5)

In [290]:
y_preds

array([2., 2., 4., 1., 1., 1., 1., 1., 2., 1., 4., 4., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 4., 1., 1., 1., 3., 1., 4., 2., 1., 1.,
       1., 1., 1., 3., 4., 3., 2., 1., 1., 3., 1., 1., 1., 1., 1., 1., 3.,
       1., 2., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 4., 2., 3.,
       1., 1., 1., 3., 1., 1., 4., 1., 3., 2., 1., 1., 4., 2., 2., 1., 1.,
       3., 2., 4., 1., 3., 1., 1., 1., 1., 1., 4., 1., 1., 4., 3., 1., 1.,
       1., 2., 1., 1., 1., 2., 1., 2., 3., 1., 1., 1., 1., 3., 1., 1., 1.,
       1., 3., 1., 2., 1., 1., 1., 1., 2., 2., 3., 1., 1., 1., 1., 3., 2.,
       1., 1., 1., 1., 1., 3., 2., 1., 1., 3., 4., 1., 1., 1., 3., 1., 1.,
       1., 1., 1., 2., 1., 1., 1., 1., 1., 4., 1., 1., 1., 3., 1., 2., 2.,
       1., 3., 3., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.,
       4., 1., 3., 1., 4., 1., 1., 2., 2., 1., 1., 2., 1., 1., 1., 1., 2.,
       2., 1., 1., 1., 1., 4., 1., 1., 3., 1., 2., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2.

In [291]:
# make predictions
y_preds = classifier.predict(X_test5)
y_preds = [int(i) for i in y_preds]
assert len(y_preds) == len(test)

# create submission csv

sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['category'] = y_preds
sample_submission.to_csv('team_whiskey_submission2.csv', index=False)

#score = 0.95

In [300]:
classifier = LogisticRegression(solver='lbfgs')

In [301]:
classifier.fit(X_train4, y_train)

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceW

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [303]:
# make predictions
y_preds = classifier.predict(X_test5)
y_preds = [int(i) for i in y_preds]
assert len(y_preds) == len(test)

# create submission csv

sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['category'] = y_preds
sample_submission.to_csv('team_whiskey_submission4.csv', index=False)